# 特徵工程 (Feature Engineering)
負責算 MA, RSI, MACD (Feature Engineering)

In [1]:
import pandas as pd
import numpy as np

# 造一個假想的股價，連續 30 天
# 假裝股價從 10 元慢慢漲到 40 元，中間波動一下
dates = pd.date_range(start='2024-01-01', periods=30)
prices = [10, 11, 12, 11, 13, 15, 14, 16, 18, 20, 
          22, 24, 23, 25, 28, 30, 29, 31, 33, 35, 
          34, 36, 38, 40, 39, 41, 43, 42, 45, 48]

# 建立一個測試用的 DataFrame
df_test = pd.DataFrame({
    'date': dates,
    '1101_close': prices,      # 假裝是台泥
    '2330_close': prices       # 假裝是台積電 (複製一份來測試多欄位)
})

print("🧪 測試資料準備完成！")
print(df_test.head())

🧪 測試資料準備完成！
        date  1101_close  2330_close
0 2024-01-01          10          10
1 2024-01-02          11          11
2 2024-01-03          12          12
3 2024-01-04          11          11
4 2024-01-05          13          13



1. 相對強弱指標 (RSI)


In [ ]:
def calculate_rsi(series, period=14):
    """
    計算 RSI (Relative Strength Index)
    
    原理：
        測量股價漲跌的「速度」與「變動幅度」。
        範圍為 0-100。
        - > 70: 超買 (Overbought)，可能過熱，小心回檔。
        - < 30: 超賣 (Oversold)，可能過冷，有機會反彈。
    
    參數：
        series (pd.Series): 收盤價數列
        period (int): 計算週期，預設 14 天 (標準參數)
        
    回傳:
        pd.Series: 計算好的 RSI 數值
    """
    # 1. 計算每日價格變動 (今天 - 昨天)
    delta = series.diff()
    
    # 2. 區分「漲幅」與「跌幅」
    # 如果 delta > 0，取 delta 值，否則為 0
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    # 如果 delta < 0，取 -delta (把負數轉正)，否則為 0
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    # 3. 計算 RS (相對強弱值) = 平均漲幅 / 平均跌幅
    rs = gain / loss

    # 4. 正規化為 RSI (0~100)
    # 公式: 100 - (100 / (1 + RS))
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# --- 🧪 立即測試 RSI ---
print("正在測試 RSI...")
# 我們拿剛剛造的 1101_close 來測
rsi_result = calculate_rsi(df_test['1101_close'], period=14) 
print(rsi_result.tail(10)) # 印出最後 10 天看看有沒有數值

2. 平滑異同移動平均線 (MACD)

In [2]:
def calculate_macd(series, fast=12, slow=26, signal=9):
    """
    計算 MACD (Moving Average Convergence Divergence)
    
    原理：
        利用兩條不同速度的均線，判斷股價的「趨勢」與「動能」。
        
    參數：
        fast (int): 快線週期 (預設 12 天)
        slow (int): 慢線週期 (預設 26 天)
        signal (int): 訊號線平滑週期 (預設 9 天)
        
    回傳 (Tuple):
        dif (快慢線差): 本體，判斷多空趨勢 (正=多, 負=空)。
        dem (訊號線): dif 的平均線，用來判斷黃金/死亡交叉。
        osc (柱狀圖): 動能強弱，柱子變長代表趨勢加速，變短代表背離(沒力了)。
    """
    # 1. 計算 EMA (指數移動平均) - 越近期的股價權重越重
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    # 2. 計算 DIF (快線 - 慢線)
    dif = ema_fast - ema_slow
    # 3. 計算 DEM (訊號線) - 對 DIF 再做一次平滑
    dem = dif.ewm(span=signal, adjust=False).mean()
    # 4. 計算 OSC (柱狀圖/震盪量) - 兩條線的距離
    osc = dif - dem
    
    return dif, dem, osc

# --- 🧪 立即測試 MACD ---
print("正在測試 MACD...")
dif, dem, osc = calculate_macd(df_test['1101_close'])

# 檢查一下是不是真的有三個東西跑出來
print("DIF 前5筆:", dif.tail(5).values)
print("OSC 前5筆:", osc.tail(5).values)

正在測試 MACD...
DIF 前5筆: [6.75587628 6.94618689 6.93635975 7.08892982 7.3669955 ]
OSC 前5筆: [0.86547853 0.84463132 0.66784334 0.65633073 0.74751712]


3. 主整合函式 (Manager)

In [4]:
import re
import pandas as pd
import numpy as np

def add_technical_indicators(df):
    """
    特徵工程主控台 (升級版)
    
    功能：
        自動偵測 DataFrame 中的收盤價欄位，並計算：
        1. 基礎指標: MA, RSI, MACD
        2. 進階指標: 乖離率 (Bias), 收益率 (Return), 波動率 (Volatility), 布林位置 (BBP)
        
    輸入：
        df (pd.DataFrame): 原始資料表，必須包含 'close' 相關欄位
        
    輸出：
        pd.DataFrame: 加上特徵後的新資料表
    """
    df = df.copy()
    
    # 自動抓取所有包含 'close' 的欄位名稱 (不分大小寫)
    close_cols = [c for c in df.columns if 'close' in c.lower()]
    print(f"📊 偵測到 {len(close_cols)} 支股票，開始計算全套技術指標...")

    for col in close_cols:
        # 使用 Regex 解析股票代號 (例如 '1101_close' -> '1101')
        match = re.search(r'\d+', col)
        if match:
            prefix = match.group() # 抓出 '1101'
        else:
            print(f"⚠️ 跳過欄位 {col}：找不到股票代號")
            continue
        
        # -------------------------------------------------------
        # 1. 基礎趨勢指標
        # -------------------------------------------------------
        
        # --- A. MA (移動平均線) ---
        ma_5 = df[col].rolling(window=5).mean()
        ma_20 = df[col].rolling(window=20).mean() # 新增 MA20 給布林通道用
        
        df[f'{prefix}_MA_5'] = ma_5

        # --- B. RSI (相對強弱指標) ---
        # 注意：請確保您的檔案上方有定義 calculate_rsi 函式，否則會報錯
        try:
            df[f'{prefix}_RSI'] = calculate_rsi(df[col], period=14)
        except NameError:
            print("⚠️ 警告：找不到 calculate_rsi 函式，跳過 RSI 計算")
        
        # --- C. MACD (趨勢指標) ---
        # 注意：請確保您的檔案上方有定義 calculate_macd 函式
        try:
            dif, dem, osc = calculate_macd(df[col])
            df[f'{prefix}_MACD_DIF'] = dif
            df[f'{prefix}_MACD_DEM'] = dem
            df[f'{prefix}_MACD_OSC'] = osc
        except NameError:
            print("⚠️ 警告：找不到 calculate_macd 函式，跳過 MACD 計算")

        # -------------------------------------------------------
        # 2. 新增進階指標 (New Features) 🔥
        # -------------------------------------------------------

        # --- D. 乖離率 (Bias Ratio) ---
        # 意義：股價離均線太遠會「回歸」。正乖離太大賣出，負乖離太大買進。
        df[f'{prefix}_Bias_5'] = (df[col] - ma_5) / ma_5
        df[f'{prefix}_Bias_20'] = (df[col] - ma_20) / ma_20

        # --- E. 收益率 (Returns) ---
        # 意義：動能指標，看昨今兩天的漲跌幅
        df[f'{prefix}_Return_1'] = df[col].pct_change()
        
        # --- F. 波動率 (Volatility) ---
        # 意義：風險指標，計算過去 20 天漲跌幅的標準差
        df[f'{prefix}_Vol_20'] = df[col].pct_change().rolling(window=20).std()
        
        # --- G. 布林通道位置 (Bollinger Band Position / %B) ---
        # 意義：股價在通道內的相對位置。 >1 代表突破上軌(超強勢)，<0 代表跌破下軌(超弱勢)
        std_20 = df[col].rolling(window=20).std()
        upper_band = ma_20 + (2 * std_20)
        lower_band = ma_20 - (2 * std_20)
        
        # (股價 - 下軌) / (上軌 - 下軌)
        # 加 1e-9 是為了防止除以 0 報錯
        df[f'{prefix}_BBP'] = (df[col] - lower_band) / (upper_band - lower_band + 1e-9)

    
    # 最後處理：補值
    # 因為 rolling(20) 會導致前 20 筆資料是 NaN，這裡用 0 填補
    df = df.fillna(0)
    
    return df

# --- 🧪 立即整合測試 ---

print("正在執行最終整合測試...")

df_final = add_technical_indicators(df_test)



print("\n✅ 測試成功！看看產生了哪些新欄位：")

print(df_final.columns.tolist())

print("\n內容預覽：")

print(df_final.tail())

正在執行最終整合測試...
📊 偵測到 2 支股票，開始計算全套技術指標...
⚠️ 警告：找不到 calculate_rsi 函式，跳過 RSI 計算
⚠️ 警告：找不到 calculate_rsi 函式，跳過 RSI 計算

✅ 測試成功！看看產生了哪些新欄位：
['date', '1101_close', '2330_close', '1101_MA_5', '1101_MACD_DIF', '1101_MACD_DEM', '1101_MACD_OSC', '1101_Bias_5', '1101_Bias_20', '1101_Return_1', '1101_Vol_20', '1101_BBP', '2330_MA_5', '2330_MACD_DIF', '2330_MACD_DEM', '2330_MACD_OSC', '2330_Bias_5', '2330_Bias_20', '2330_Return_1', '2330_Vol_20', '2330_BBP']

內容預覽：
         date  1101_close  2330_close  1101_MA_5  1101_MACD_DIF  \
25 2024-01-26          41          41       38.8       6.755876   
26 2024-01-27          43          43       40.2       6.946187   
27 2024-01-28          42          42       41.0       6.936360   
28 2024-01-29          45          45       42.0       7.088930   
29 2024-01-30          48          48       43.8       7.366995   

    1101_MACD_DEM  1101_MACD_OSC  1101_Bias_5  1101_Bias_20  1101_Return_1  \
25       5.890398       0.865479     0.056701      0.423611    